In [1]:
import requests
import time
import json
import threading
import queue
import random
import datetime
import os
import urllib3
import pickle

from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from collections import defaultdict
from pprint import pprint

from config.assistant import *

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
#from config.site_config import *

In [9]:
# searchType=search
# searchword=컴퓨터비전         # 검색어
# loc_mcd=101000,102000         # 지역, 101000(서울),102000(경기)
# company_cd=0,1,2,3,4,5,6,7    # 채용제공구분 / 9(파견/대행), 10(헤드헌팅)
# search_optional_item=y
# search_done=y
# panel_count=y
# recruitPage=1                 # 현재 페이지뷰
# recruitSort=relation          # 정렬방법
# recruitPageCount=100          # 표시 개수
# inner_com_type=
# show_applied=                 # 지원한 공고 표시
# quick_apply=                  # 빠른 입사지원
# except_read=                  # 조회한 페이지 제외
# ai_head_hunting=
# mainSearch=n

In [10]:
search_words = ["컴퓨터비전", 'COMPUTER VISION',
                "영상처리", "IMAGE PROCESSING",
                "딥러닝", "DEEP LEARNING", 
                "머신러닝", "MACHINE LEARNING",
                "이미지인식", "IMAGE RECOGNITION",
                "이미지분석", "IMAGE ANALYSIS",
                "VISION AI",
                "OPENCV"
               ]

search_words = ["이미지인식"]
page_view_items = 100
company_cd='0,1,2,3,4,5,6,7'
base_link = 'https://www.saramin.co.kr'
job_group = {}


In [11]:
# 저장된 키워드 별 검색
for search_word in search_words:
    tic()
    item_count = 0
    current_page = 1
    
    # 검색 결과 리스트 페이지 별 확인
    search_loop = True
    while(search_loop):
        search_link = f'https://www.saramin.co.kr/zf_user/search/recruit'\
            + f'?search_area=main'\
            + f'&search_done=y'\
            + f'&search_optional_item=n'\
            + f'&searchType=search'\
            + f'&recruitSort=relation'\
            + f'&searchword={search_word}'\
            + f'&recruitPage={current_page}'\
            + f'&recruitPageCount={page_view_items}'\
            + f'&company_cd={company_cd}'\
            + f'&mainSearch=y'

        response = requests.get(search_link, verify=False)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        if len(soup.find_all('div', attrs={'class': 'info_no_result'})) == 1:
            search_loop = False
        else:
            current_page += 1

            page_items = len(soup.find_all('h2', attrs={'class': 'job_tit'}))

            # 해당 페이지 리스트 (1 ~ 100)
            for i in range(1, page_items+1):
                elements = soup.select(f'div.content > div:nth-child({i})')[0]

                # 채용공고명
                str_title = elements.find('a', attrs={'class': 'data_layer'})['title']

                # 회사명
                try:
                    str_company1 = elements.find('a', attrs={'class': 'track_event data_layer'}).text
                    #str_company = str_company.replace(' ', '').replace('\n', '').replace('(주)', '')
                    str_company2 = str_company1.replace(' ', '')
                    str_company3 = str_company2.replace('\n', '')
                    str_company = str_company3.replace('(주)', '')
                except:
                    print(str_company1)
                    print(str_company2)
                    print(str_company3)
                    print(str_company)

                # 채용공고 링크
                str_title_link = base_link + elements.find('a', attrs={'class': 'data_layer'})['href']

                # 회사 링크
                str_company_link = base_link + elements.find('a', attrs={'class': 'track_event data_layer'})['href']

                # Dictionary에 저장
                if str_company in job_group:    # 회사명 - 기존에 있으면
                    for company in job_group[str_company]:
                        if company['title'] == str_title:    # 채용공고명 - 기존에 있으면
                            break
                    else:
                        job_group[str_company].append({'company': str_company, \
                                                       'title': str_title, \
                                                       'title_link': str_title_link, \
                                                       'company_link': str_company_link, \
                                                       'input_date': datetime.date.today().isoformat(), \
                                                       'status': 'wait'
                                                      })

                else:    # 회사명 - 신규진입
                    job_group[str_company] = []
                    job_group[str_company].append({'company': str_company, \
                                                   'title': str_title, \
                                                   'title_link': str_title_link, \
                                                   'company_link': str_company_link, \
                                                   'input_date': datetime.date.today().isoformat(), \
                                                   'status': 'wait'
                                                  })

                item_count += 1
                
    print(f'{search_word}: {item_count}')
    toc()

이미지인식: 214
Elapsed time is 12.548541784286499 seconds.


In [2]:
len(job_group)

NameError: name 'job_group' is not defined

In [12]:
# 머신러닝: 1450
# MACHINE LEARNING: 1445
# 딥러닝: 1158
# DEEP LEARNING: 1158
# 컴퓨터비전: 1120
# 영상처리: 1037
# COMPUTER VISION: 964
# 이미지분석: 714
# VISION AI: 657
# IMAGE PROCESSING: 295
# OPENCV: 232
# 이미지인식: 222
# IMAGE ANALYSIS: 21
# IMAGE RECOGNITION: 12

# 9365
# job_group 2259

In [13]:
# JSON 파일 쓰기
file_path = "config/job_group.json"

tf = open(file_path, 'w')
json.dump(job_group, tf)
tf.close()

In [2]:
# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)

In [3]:
print(len(job_group))

24


In [4]:
copy_group = {}

In [5]:
input_status = 5
if input_status == 1:      status = 'all'
elif input_status == 2:    status = 'wait'
elif input_status == 3:    status = 'save'
elif input_status == 4:    status = 'hold'
elif input_status == 5:    status = 'close'
        
# ALL   : 1
# WAIT  : 2
# SAVE  : 3
# HOLD  : 4
# CLOSE : 5

def filter_status_func():
    try:
        copy_group[companys].append(job_group[companys][idx])
    except:
        copy_group[companys] = []
        copy_group[companys].append(job_group[companys][idx])
    
for companys in job_group:
    for idx, job_list in enumerate(job_group[companys]):
        if job_group[companys][idx]['status'] == status:    # ALL
            test_func()
        elif job_group[companys][idx]['status'] == status:    # WAIT
            test_func()
        elif job_group[companys][idx]['status'] == status:    # SAVE
            test_func()
        elif job_group[companys][idx]['status'] == status:    # HOLD
            test_func()
        elif job_group[companys][idx]['status'] == status:    # CLOSE
            test_func()   

In [6]:
pprint(copy_group)

{'스텔란티스코리아': [{'company': '스텔란티스코리아',
               'company_link': 'https://www.saramin.co.kr/zf_user/company-info/view?csn=QWZyVlFHKy9RQkNRQmhSVXRDWnJ5UT09',
               'input_date': '2022-08-28',
               'status': 'close',
               'title': '[수입차 Jeep/Peugeot] Product Planning (대리-과장급)',
               'title_idx': 1,
               'title_link': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=search&rec_idx=43852302&location=ts&searchword=IMAGE+ANALYSIS&searchType=search&paid_fl=n&search_uuid=1fea4ee1-80dd-49cb-a985-e7bbcf9629ff'}],
 '제이엘케이': [{'company': '제이엘케이',
            'company_link': 'https://www.saramin.co.kr/zf_user/company-info/view?csn=dGJHczlpN0NvYWNkaEhUbGRXZXdKUT09',
            'input_date': '2022-08-28',
            'status': 'close',
            'title': '[JLK] 인공지능(AI)·빅데이터연구원 채용(병역특례가능)',
            'title_idx': 0,
            'title_link': 'https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=search&rec_idx=43622025&locati

In [3]:
# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)
print(len(job_group))

4843


In [6]:
send_job_group = {}
if len(job_group) > 100:
    print("11")

11


In [19]:
for idx, title_name in enumerate(job_group):
    print(idx)
    send_job_group[title_name] = job_group[title_name]
    break

0


In [13]:
send_job_group["팅크웨어"][0]['status'] = "close"

In [14]:
send_job_group["팅크웨어"][0]['status']

'close'

In [15]:
job_group["팅크웨어"][0]['status']

'close'